In [1]:
!pip install jupyter-dash
!pip install setuptools
!pip install packaging
!pip install pandas dash
!pip install httpx plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 17.9 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3


In [2]:
# Import required libraries
import pandas as pd
import dash
# import dash_html_components as html
from dash import html
# import dash_core_components as dcc
from dash import dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [3]:
# Read the airline data into pandas dataframe
url = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv"

spacex_df = pd.read_csv(url)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

print(max_payload)
print(min_payload)
spacex_df.head()

9600.0
0.0


,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [4]:
# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),

                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                              options=[
                                                  {'label': 'All Sites', 'value': 'ALL'},
                                                  {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                                                  {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                                                  {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'},
                                                  {'label': 'CCAFS SLC-40', 'value': 'CCAFS SLC-40'}
                                              ],
                                              value='ALL',
                                              placeholder="Select launch site here",
                                              searchable=True
                                ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',100: '100'},
                                                value=[min_payload, max_payload]),
                                #dcc.RangeSlider(id='payload-slider',...)

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

In [ ]:
spacex_df.columns

Index(['Unnamed: 0', 'Flight Number', 'Launch Site', 'class',
       'Payload Mass (kg)', 'Booster Version', 'Booster Version Category'],
      dtype='object')

In [5]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
               Input(component_id='site-dropdown', component_property='value'))

def get_pie_chart(entered_site):
  if entered_site == 'ALL':
    fig = px.pie(spacex_df, values='class',
    names='Launch Site',
    title='Success rate of each launch site')
    # return fig

  else:
    site_select = spacex_df[spacex_df['Launch Site']==entered_site]
    pie_data = site_select['class'].value_counts().reset_index()
    pie_data.columns = ['Class', 'Count']
    fig = px.pie(pie_data, values='Count',
    names='Class',
    title=f'Success rate of {entered_site}')

  return fig

In [6]:
# site_select = spacex_df[spacex_df['Launch Site']=='VAFB SLC-4E']
# pie_data = site_select['class'].value_counts().reset_index()
# pie_data.columns = ['Class', 'Count']
# fig = px.pie(pie_data, values='Count',names='Class',title=f'Success rate of VAFB SLC-4E')
# fig.show()

In [7]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(
    Output(component_id='success-payload-scatter-chart', component_property='figure'),
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id='payload-slider', component_property='value')])

def selected_site(site, payload):

  min_payload, max_payload = payload
  data_filter = spacex_df[(spacex_df['Payload Mass (kg)'] >= min_payload) &
                            (spacex_df['Payload Mass (kg)'] <= max_payload)]

  if site == 'ALL':
    fig = px.scatter(data_filter, x='Payload Mass (kg)', y='class', color='Booster Version', title="Correlation between payload mass and success rate for all site")


  else:
    site_df = data_filter[data_filter['Launch Site'] == site]
    fig = px.scatter(site_df, x='Payload Mass (kg)', y='class', color='Booster Version', title=f"Correlation between payload mass and success rate of {site}")

  return fig

  # elif site == 'VAFB SLC-4E' and payload:
  #   slc_4esite = spacex_df[spacex_df['Launch Site']=='VAFB SLC-4E']
  #   fig = px.scatter(slc_4esite, x='Payload Mass', y='class', color='Booster Version', title="Correlation between payload mass and success rate")

  # elif site == 'KSC LC-39A' and payload:
  #   lc_39asite = spacex_df[spacex_df['Launch Site']=='KSC LC-39A']
  #   fig = px.scatter(lc_39asite, x='Payload Mass', y='class', color='Booster Version', title="Correlation between payload mass and success rate")

  # elif site == 'CCAFS SLC-40' and payload:
  #   slc_40site = spacex_df[spacex_df['Launch Site']=='CCAFS SLC-40']
  #   fig = px.scatter(slc_40site, x='Payload Mass', y='class', color='Booster Version', title="Correlation between payload mass and success rate")

  # else:
  #     return None

In [8]:
# Run the app
if __name__ == '__main__':
    app.run_server()

<IPython.core.display.Javascript object>